In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image = cv2.imread("chessboard.jpg")
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Step 1: Edge Detection using Canny
edges = cv2.Canny(blurred, 50, 150)

# Step 2: Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 3: Sort contours by area and find the largest contour (the chessboard)
contours = sorted(contours, key=cv2.contourArea, reverse=True)
doc_contour = None

for contour in contours:
    # Approximate the contour to a polygon
    epsilon = 0.04 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # If the approximated polygon has 4 corners, we assume it's the chessboard
    if len(approx) == 4:
        doc_contour = approx
        break

# Step 4: If we found a valid contour, apply perspective transformation
if doc_contour is not None:
    # Get the 4 corners of the detected chessboard
    pts = doc_contour.reshape(4, 2)

    # Order the points in top-left, top-right, bottom-right, bottom-left order
    pts = sorted(pts, key=lambda x: x[0])  # Sort by x-coordinate
    if pts[0][1] > pts[1][1]:
        pts[0], pts[1] = pts[1], pts[0]
    if pts[2][1] < pts[3][1]:
        pts[2], pts[3] = pts[3], pts[2]

    # Define the destination points (top-down view of the chessboard)
    width = max(np.linalg.norm(pts[2] - pts[3]), np.linalg.norm(pts[1] - pts[0]))
    height = max(np.linalg.norm(pts[1] - pts[2]), np.linalg.norm(pts[3] - pts[0]))
    dst_pts = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype="float32")

    # Step 5: Get the perspective transform matrix
    M = cv2.getPerspectiveTransform(np.float32(pts), dst_pts)

    # Apply the perspective warp to get the top-down view
    warped_image = cv2.warpPerspective(image, M, (int(width), int(height)))

    # Display the results
    plt.figure(figsize=(10, 8))
    plt.subplot(121)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.subplot(122)
    plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))
    plt.title("Warped Image (Top-down View)")
    plt.show()
else:
    print("Chessboard contour not found")
